In [12]:
import os, re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [13]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [14]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [15]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

In [16]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f86cef77690>


In [17]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [18]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

In [65]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 128
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [20]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 1.08742734e-05,  5.99184095e-05,  4.80442977e-04, ...,
          1.84724398e-04, -2.44788644e-05,  4.37384806e-05],
        [-1.05528001e-04, -1.20581244e-04,  6.91066612e-04, ...,
          2.27122466e-04, -2.34005885e-04, -2.55881168e-04],
        [-4.91161656e-04, -1.04417311e-04,  7.50367704e-04, ...,
          4.19394491e-04, -5.32279315e-04, -7.69473496e-04],
        ...,
        [ 1.29488099e-03,  1.05529011e-03, -1.58248018e-04, ...,
          2.83851265e-03, -1.05201569e-03, -4.16524650e-04],
        [ 1.64137012e-03,  1.26616575e-03, -2.09075763e-04, ...,
          3.25974938e-03, -1.41977158e-03, -3.66497057e-04],
        [ 1.92260172e-03,  1.44085602e-03, -2.79926200e-04, ...,
          3.63796996e-03, -1.71026890e-03, -3.01924476e-04]],

       [[ 1.08742734e-05,  5.99184095e-05,  4.80442977e-04, ...,
          1.84724398e-04, -2.44788644e-05,  4.37384806e-05],
        [ 5.34999126e-05,  2.87860224e-04,  8.

In [21]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [12]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [24]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he hath not dared to make a puppet of him . <end> '

---

# 제출용

In [1]:
import glob
import os
import re
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me']


In [2]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제

    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter(구분자)로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다

    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("Thissentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("I eat lunch"))

<start> this is sample sentence . <end>
<start> thissentence . <end>
<start> i eat lunch <end>


In [3]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue # 길이가 0인 문장은 건너뜁니다
    if len(sentence) > 50: continue # 길이가 너무 긴것은 건너뜁니다
    if sentence[-1] == ":": continue # 문장의 끝이 :인 문장은 건너뜁니다
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> hook <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> somethin ain t right when we talkin <end>',
 '<start> somethin ain t right when we talkin <end>']

In [4]:
import tensorflow as tf 

def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=13000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    # maxlen=15를 설정해서 토큰의 개수가 15개를 최대로 설정, 15개 초과하는 데이터는 제외
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  941    3 ...    0    0    0]
 [   2    4   94 ...   10   12    3]
 [   2   46  134 ...    0    0    0]
 ...
 [   2  198    3 ...    0    0    0]
 [   2  442    9 ...    0    0    0]
 [   2    9 1610 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f0428242a90>


In [5]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [6]:

src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])
print(len(src_input))
print(len(tgt_input))

[  2 941   3   0   0   0   0   0   0   0   0   0   0   0]
[941   3   0   0   0   0   0   0   0   0   0   0   0   0]
142443
142443


In [7]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True)

print("Source Train:", enc_train.shape) # 소스 문장
print("Target Train:", dec_train.shape) # 타겟 문장

Source Train: (113954, 14)
Target Train: (113954, 14)


In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 13000개와, 여기 포함되지 않은 0:<pad>를 포함하여 13001개

dataset1 = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset1 = dataset1.batch(BATCH_SIZE, drop_remainder=True)
print(dataset1)

dataset2 = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
dataset2 = dataset2.batch(BATCH_SIZE, drop_remainder=True)
print(dataset2)

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>
<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>


In [67]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 128
"""
embedding_size: 워드벡터의 차원수 즉, 단어가 추상적으로 표현되는 크기.
> 입력 텐서에는 단어 사전의 인덱스가 들어 있고, Embedding 레이어는 이 인덱스 값을 해당 인덱스의 워드 벡터로 바꿔 줍니다. 
이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현(representation)으로 사용
embedding_size가 너무 커도 너무 작아도 문제 --> 여기서는 128로 설정
"""
hidden_size = 1536
"""
hidden_size: LSTM 레이어의 hidden state 의 차원수.
> 얼마나 많은 일꾼을 둘 것인지? 너무 많다면 배가 산으로 가고 너무 적으면 성능이 낮다. 여기서는 1536로 적당히 설정
"""
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [68]:
for src_sample, tgt_sample in dataset1.take(1): break
print(model(src_sample))

tf.Tensor(
[[[ 2.20218877e-04  8.04887459e-05 -6.37062913e-05 ... -6.80291560e-05
    1.17088974e-04 -9.67511660e-05]
  [ 3.26413574e-04  2.21211958e-04 -9.19321319e-05 ... -7.76991292e-05
    1.39612021e-04 -6.19124039e-05]
  [ 3.57318611e-04  3.09291645e-04 -1.73720895e-04 ... -2.28864301e-04
    7.16243594e-05  7.26423559e-06]
  ...
  [ 2.54288370e-05  8.71402968e-04 -3.04060486e-05 ...  5.16583154e-04
   -1.09667715e-03  4.44058358e-04]
  [ 7.37406572e-06  1.10738317e-03  8.65218972e-05 ...  5.07884950e-04
   -1.26956648e-03  5.06662123e-04]
  [ 6.54198402e-06  1.34107040e-03  1.68208062e-04 ...  3.94971285e-04
   -1.54239393e-03  5.24863077e-04]]

 [[ 2.20218877e-04  8.04887459e-05 -6.37062913e-05 ... -6.80291560e-05
    1.17088974e-04 -9.67511660e-05]
  [ 4.05864121e-04  2.86609109e-04 -1.48143721e-04 ... -8.84152032e-05
    2.03907344e-04 -1.42528603e-04]
  [ 5.69025287e-04  5.64218208e-04 -2.01871677e-04 ... -5.81187305e-05
    2.58521730e-04 -1.46547492e-04]
  ...
  [ 9.772131

In [69]:
#Loss
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, 
              optimizer=optimizer,
              metrics=['accuracy'])


model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset1, epochs=10, validation_data=dataset2)

Epoch 1/10
445/445 [==============================] - 275s 613ms/step - loss: 3.9140 - val_loss: 3.0113
Epoch 2/10
445/445 [==============================] - 275s 619ms/step - loss: 2.9325 - val_loss: 2.8148
Epoch 3/10
445/445 [==============================] - 274s 615ms/step - loss: 2.7379 - val_loss: 2.7037
Epoch 4/10
445/445 [==============================] - 273s 613ms/step - loss: 2.5964 - val_loss: 2.6170
Epoch 5/10
445/445 [==============================] - 274s 616ms/step - loss: 2.4699 - val_loss: 2.5460
Epoch 6/10
445/445 [==============================] - 272s 612ms/step - loss: 2.3553 - val_loss: 2.4897
Epoch 7/10
445/445 [==============================] - 274s 616ms/step - loss: 2.2531 - val_loss: 2.4446
Epoch 8/10
445/445 [==============================] - 272s 612ms/step - loss: 2.1580 - val_loss: 2.4091
Epoch 9/10
445/445 [==============================] - 276s 620ms/step - loss: 2.0649 - val_loss: 2.3736
Epoch 10/10
445/445 [==============================] - 274s 615m

# VAL_LOSS가 2.2 이하로 떨어지지 않아서 embedding_size와 hidden_size 조절

In [9]:

embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [10]:
#Loss
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, 
              optimizer=optimizer,
              metrics=['accuracy'])


model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset1, epochs=10, validation_data=dataset2)

Epoch 1/10
445/445 [==============================] - 417s 886ms/step - loss: 3.9100 - val_loss: 2.9598
Epoch 2/10
445/445 [==============================] - 398s 895ms/step - loss: 2.8731 - val_loss: 2.7538
Epoch 3/10
445/445 [==============================] - 405s 910ms/step - loss: 2.6526 - val_loss: 2.6165
Epoch 4/10
445/445 [==============================] - 396s 891ms/step - loss: 2.4644 - val_loss: 2.5050
Epoch 5/10
445/445 [==============================] - 393s 883ms/step - loss: 2.2903 - val_loss: 2.4181
Epoch 6/10
445/445 [==============================] - 401s 901ms/step - loss: 2.1269 - val_loss: 2.3461
Epoch 7/10
445/445 [==============================] - 396s 889ms/step - loss: 1.9707 - val_loss: 2.2838
Epoch 8/10
445/445 [==============================] - 394s 885ms/step - loss: 1.8205 - val_loss: 2.2332
Epoch 9/10
445/445 [==============================] - 394s 885ms/step - loss: 1.6745 - val_loss: 2.1875
Epoch 10/10
445/445 [==============================] - 396s 888m

In [13]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

### 에포크10인데도 시간이 엄청 오래걸렸고 자연어처리는 어렵다는것을 다시한번 느꼈다